In [1]:
using Laplacians

objc[17563]: Class TKApplication is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[17563]: Class TKMenu is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[17563]: Class TKContentView is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.
objc[17563]: Class TKWindow is implemented in both /Users/serbanstan/anaconda/lib/libtk8.5.dylib and /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk. One of the two will be used. Which one is undefined.


In [2]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [3]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [4]:
graphp = "../graphs/se_1/"

"../graphs/se_1/"

In [5]:
a = readFromFile(graphp * "graph.mtx");

In [6]:
tree = readFromFile(graphp * "tree.mtx");

In [7]:
x = readFromFile(graphp * "x.vec");

In [8]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [9]:
@time F = treeSolver(tree)
@time F(b);

  0.278817 seconds (281.05 k allocations: 11.975 MB, 2.33% gc time)
  0.175479 seconds (131.66 k allocations: 5.323 MB, 4.03% gc time)


In [12]:
# a run on standard Float64 data types
numIts = 50000;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=false);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

 37.637860 seconds (675.89 M allocations: 18.379 GB, 14.80% gc time)


In [14]:
numIts = 10000
for hp in [64,128,1024]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end

Working on iteration 10 with a-norm error 3469.5633275243113
Working on iteration 20 with a-norm error 1573.8510088944831
Working on iteration 30 with a-norm error 1023.6216884260041
Working on iteration 40 with a-norm error 750.0182473212516
Working on iteration 50 with a-norm error 583.6278334786532
Working on iteration 60 with a-norm error 470.551956430909
Working on iteration 70 with a-norm error 398.95384818775875
Working on iteration 80 with a-norm error 336.9921477801604
Working on iteration 90 with a-norm error 289.855983018558
Working on iteration 100 with a-norm error 244.09984751902323
Working on iteration 110 with a-norm error 205.06496340374156
Working on iteration 120 with a-norm error 184.92150736718057
Working on iteration 130 with a-norm error 162.4514214734542
Working on iteration 140 with a-norm error 142.47371169151583
Working on iteration 150 with a-norm error 131.2915223215322
Working on iteration 160 with a-norm error 120.7909483704082
Working on iteration 170 wi

In [ ]:
?pcgV

In [ ]:
typeof(laHP)